In [2]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np

class PointCloudDataset(Dataset):
    def __init__(self, root_dir):
        self.points = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dir))
        class_to_idx = {self.classes[i]: i for i in range(len(self.classes))}

        for idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for file_name in os.listdir(class_dir):
                if file_name.lower().endswith('.txt'):
                    file_path = os.path.join(class_dir, file_name)
                    point_cloud = np.loadtxt(file_path)
                    self.points.append(point_cloud)
                    self.labels.append(class_to_idx[class_name])

    def __len__(self):
        return len(self.points)

    def __getitem__(self, idx):
        return torch.tensor(self.points[idx]), self.labels[idx]


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader, random_split

class PointNet(nn.Module):
    def __init__(self, num_classes=4):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.fc1 = nn.Linear(128, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.adaptive_max_pool1d(x, 1).squeeze(-1)
        x = F.relu(self.bn3(self.fc1(x)))
        x = F.relu(self.bn4(self.fc2(x)))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


In [4]:
# Training Parameters
num_epochs = 30
batch_size = 32
learning_rate = 0.001

# Prepare Dataset
dataset = PointCloudDataset(root_dir=r"C:\Users\Muhammad Hassan\Desktop\fraunhofer\Training Datasets\Simple Shapes Dataset")
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Model, Loss, Optimizer
model = PointNet(num_classes=4)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    for i, (points, labels) in enumerate(train_loader):
        points = points.transpose(1, 2).float()  # Reshape to (batch_size, 3, num_points)
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Validation Phase
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for points, labels in val_loader:
            points = points.transpose(1, 2).float()
            outputs = model(points)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')


Epoch [1/30], Step [10/100], Loss: 1.2002
Epoch [1/30], Step [20/100], Loss: 1.0452
Epoch [1/30], Step [30/100], Loss: 0.9865
Epoch [1/30], Step [40/100], Loss: 0.7612
Epoch [1/30], Step [50/100], Loss: 0.7346
Epoch [1/30], Step [60/100], Loss: 0.7286
Epoch [1/30], Step [70/100], Loss: 0.5647
Epoch [1/30], Step [80/100], Loss: 0.6495
Epoch [1/30], Step [90/100], Loss: 0.6282
Epoch [1/30], Step [100/100], Loss: 0.5261
Validation Accuracy: 53.62%
Epoch [2/30], Step [10/100], Loss: 0.4165
Epoch [2/30], Step [20/100], Loss: 0.5876
Epoch [2/30], Step [30/100], Loss: 0.3909
Epoch [2/30], Step [40/100], Loss: 0.3844
Epoch [2/30], Step [50/100], Loss: 0.5096
Epoch [2/30], Step [60/100], Loss: 0.4047
Epoch [2/30], Step [70/100], Loss: 0.4277
Epoch [2/30], Step [80/100], Loss: 0.6509
Epoch [2/30], Step [90/100], Loss: 0.4104
Epoch [2/30], Step [100/100], Loss: 0.3401
Validation Accuracy: 74.62%
Epoch [3/30], Step [10/100], Loss: 0.4870
Epoch [3/30], Step [20/100], Loss: 0.4377
Epoch [3/30], Step

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader, random_split
import tkinter as tk
from tkinter import filedialog


# Saving the trained model
root = tk.Tk()
root.withdraw()  # Hide the main window
save_path = filedialog.asksaveasfilename(
    title='Save Trained Model',
    filetypes=[('PyTorch Model', '*.pth')],
    defaultextension=[('PyTorch Model', '*.pth')]
)

if save_path:
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")
else:
    print("Model save operation cancelled.")


Model saved to C:/Users/Muhammad Hassan/Desktop/fraunhofer/CAD Dataset Maker algoritham/Trained 3.pth


In [19]:
import torch
from tkinter import filedialog
import tkinter as tk

def load_model(model_class, num_classes=4, model_path=None):
    if model_path is None:
        root = tk.Tk()
        root.withdraw()
        model_path = filedialog.askopenfilename(title='Select Trained Model', filetypes=[('PyTorch Model', '*.pth')])
    
    if model_path:
        model = model_class(num_classes=num_classes)
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
        model.eval()
        return model
    else:
        print("No model selected")
        return None

# Usage
model = load_model(PointNet, num_classes=4)


In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tkinter as tk
from tkinter import filedialog
import open3d as o3d
import numpy as np


class PointNet(nn.Module):
    def __init__(self, num_classes=4):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.fc1 = nn.Linear(128, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)

    def forward(self, x):
        
        x = F.relu(self.bn1(self.conv1(x)))
        
        x = F.relu(self.bn2(self.conv2(x)))
        
        x = F.adaptive_max_pool1d(x, 1).squeeze(-1)
        
        x = F.relu(self.bn3(self.fc1(x)))
        
        x = F.relu(self.bn4(self.fc2(x)))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1) 


def stl_to_point_cloud(stl_file_path, number_of_points=2048):
    mesh = o3d.io.read_triangle_mesh(stl_file_path)
    if not mesh.is_watertight():
        mesh = mesh.remove_duplicated_triangles()
        mesh = mesh.remove_duplicated_vertices()
        mesh = mesh.remove_degenerate_triangles()
    point_cloud = mesh.sample_points_poisson_disk(number_of_points)
    return np.asarray(point_cloud.points)

# Prediction function
def predict(model, point_cloud):
    point_cloud = np.asarray(point_cloud)
    point_cloud = torch.from_numpy(point_cloud).float()
    point_cloud = point_cloud.transpose(0, 1).unsqueeze(0)

    model.eval()
    with torch.no_grad():
        outputs = model(point_cloud)
        _, predicted = torch.max(outputs, dim=1)  # Ensure this is the right dimension
        return predicted.item()  # This should return a scalar value


def select_stl_and_predict(model):
    root = tk.Tk()
    root.withdraw()
    stl_file_path = filedialog.askopenfilename(title='Select STL File', filetypes=[('STL Files', '*.stl')])

    if stl_file_path:
        point_cloud = stl_to_point_cloud(stl_file_path)
        prediction = predict(model, point_cloud)
        print("Predicted class:", prediction)

        # Define a mapping from class index to manufacturing processes
        manufacturing_processes = {
            0: "Holes are Detected. We need Drilling and Boring",
            1: "Chamfer or Fillets are detected. We need Milling and Turning",
            2: "Chamfer or Fillets are detected. We need Milling and Turning",
            3: "Pockets are detected.We need Milling and Boring"
        }

        # Use the prediction to get the respective manufacturing process
        process = manufacturing_processes.get(prediction, "Unknown class")
        print("Recommended manufacturing processes:", process)
    else:
        print("No STL file selected")

# Execute the prediction using the pre-trained model
if model:
    select_stl_and_predict(model)


Predicted class: 2
Recommended manufacturing processes: Chamfer or Fillets are detected. We need Milling and Turning
